In [2]:
"""
批量读取每个被试文件夹下的.csv特征集
对每个人的数据进行单独归一化再merge
"""
import pandas as pd
import sys
sys.path.append('..')
from utils.utils_iFEMG_feature import df_save_csv, df_norm, df_merge_colname

In [3]:
"""
获取文件路径下多个文件夹中.csv格式的特征集

1. 文件夹路径
2. 肌肉bi tri
"""
# 文件夹路径
folder_path = r"E:\fuckinggggg\paper_iFEMG_4th\data"
subject_name_list = ['241114s1', '241128s2', '241204s3', '241204s4', '241206s5', '241206s6', '241225s7', '241225s8']


file_df_list = []
for name in subject_name_list:
    file_path = folder_path + '\\' + name + '\\' + "fea_set.csv"
    try:
        file_df_list.append(pd.read_csv(file_path, header = [0], index_col = None))
    except FileNotFoundError:
        print(f"文件 {file_path} 未读取")
    pass

print(f"文件数: {len(file_df_list)}")


文件数: 8


In [5]:
# 生成需要归一化的列名list
muscle_name_lst = ['RF', 'LF', 'SarT', 'BF', 'SemiT', 'VM']
feature_name_lst = ['initial_pressure_ave', 'FMG', 'mav', 'rms', 'wave_length', 'zero_crossing', 'slope_sign_change', 'mean_freq', 'mean_power_freq']
col_lst = []
for muscle in muscle_name_lst:
    for fea in feature_name_lst:
        col_lst.append(muscle + fea)

len(col_lst)

54

In [6]:
"""
对每名被试信号特征分别单独归一化

NOTE:
    - 需要修改保存文件的文件名！！
"""

# 归一化某特征
normed_df_list = [] 
err_df_index = 0    # 如果有key error发生报错，报告所在df的index
for df in file_df_list:
    # 对指定的列进行归一化
    try:
        h_df = df_norm(dataframe=df[df['side'] == 'healthy'], col_name=col_lst, method='z_score')
        a_df = df_norm(dataframe=df[df['side'] == 'affected'], col_name=col_lst, method='z_score')
        normed_df = pd.concat([a_df, h_df], axis = 0, ignore_index=True)    # 纵向合并健侧和患侧
        normed_df_list.append(normed_df)
    except KeyError:
        print(f"index {err_df_index} 存在错误")
        pass
    err_df_index = err_df_index + 1
    pass
# 合并dataframe
final_data_df = pd.concat(normed_df_list, axis = 0, ignore_index = True)

In [7]:
final_data_df

,subject,side,motion_type,label,RFinitial_pressure_ave,RFFMG,RFmav,RFrms,RFwave_length,RFzero_crossing,...,SemiTmean_power_freq,VMinitial_pressure_ave,VMFMG,VMmav,VMrms,VMwave_length,VMzero_crossing,VMslope_sign_change,VMmean_freq,VMmean_power_freq
0,241114s1,affected,qian,0.0,-1.467246,0.689316,0.984964,1.011946,1.348907,0.661664,...,-1.349457,-1.153392,-1.742492,0.749309,0.372548,0.760631,-0.049540,-1.470970,7.417798,7.621861
1,241114s1,affected,qian,0.0,-1.467246,0.723803,3.542457,3.002762,2.820437,-1.597161,...,-1.518464,-1.153392,-1.717675,-0.478887,-0.497510,-0.524464,-0.667874,-0.945640,0.000410,-0.095876
2,241114s1,affected,qian,0.0,-1.467246,0.672402,1.195944,0.967188,1.161561,-0.687569,...,-1.626836,-1.153392,-1.140261,0.125599,0.077141,0.330121,-0.524945,-0.440301,0.221353,0.095713
3,241114s1,affected,qian,0.0,-1.467246,0.736716,0.768713,0.451959,0.772827,-1.109573,...,-1.822230,-1.153392,-1.220583,-0.128541,-0.170387,-0.039458,-0.626717,-0.718020,0.101649,-0.041335
4,241114s1,affected,qian,0.0,-1.467246,0.684170,0.416196,0.211935,0.336151,-0.987257,...,-1.940627,-1.153392,-1.216346,0.182051,0.118398,0.217389,-0.566203,-0.568398,0.082630,-0.101565
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519,241225s8,healthy,hou,3.0,1.739324,-0.817715,-1.188796,-1.254183,-1.044343,1.531841,...,2.881042,2.165705,1.660043,1.247213,0.995066,1.143074,-1.594111,-1.540788,-0.917381,-1.197026
520,241225s8,healthy,hou,3.0,1.739324,1.754620,-0.309457,-0.461234,-0.335932,-0.461202,...,-0.319508,2.165705,1.762403,2.555761,2.219612,2.475992,-1.427012,-1.160936,-0.117344,-0.025819
521,241225s8,healthy,hou,3.0,1.739324,1.224749,-0.836512,-1.045724,-0.651385,0.375968,...,1.892830,2.165705,1.836548,0.352165,-0.597299,0.316607,-1.337966,-1.039247,-0.178805,-0.845473
522,241225s8,healthy,hou,3.0,1.739324,1.403272,-0.756891,-0.612523,-0.658204,0.045647,...,3.430790,2.165705,1.669868,-0.032968,-0.676959,-0.013388,-1.412566,-1.044510,-0.327425,-0.788335


In [8]:
# 保存文件
df_save_csv(final_data_df, r"E:\fuckinggggg\paper_iFEMG_4th\data\fea_normed250509.csv")

File E:\fuckinggggg\paper_iFEMG_4th\data\fea_normed250509.csv saved!
